# Exercise 3.4

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import normalized_mutual_info_score

In [38]:
# Load data
raw_birddata = pd.read_csv('data/birds2024ext.csv', sep=';', index_col=0)
pd.set_option('display.max_columns', None)
display(raw_birddata)

,group,length,wspan,weight,AR,wload,back,belly,ftype,sim,billcol,legcol,arrives,leaves,eggs,incub,ccare,biotope,diet,diver,long-billed,webbed-feet,long-legs,wading-bird,plunge-dives
species,,,,,,,,,,,,,,,,,,,,,,,,,
naurulokki,laridae,34-38,86-99,200-350,8.13,0.31,light grey,white,B,Yes,red,red,March-April,July,1-3,both,both,"lakes,sea-bays","fish,invertebrates,garbage",No,No,Yes,No,No,Yes
harmaalokki,laridae,55-65,123-148,800-1300,8.24,0.64,bluish grey,white,B,Yes,yellow,reddish,March-April,August-December,1-3,both,both,"lakes,sea-coast,marshland","fish,garbage,chicks,grain",No,No,Yes,No,No,Yes
isolokki,laridae,63-68,138-158,1000-1800,8.24,0.66,bluish grey,white,B,Yes,yellow,reddish,December,March,3,both,both,"sea-coast, harbours","fish,eggs,chicks,garbage,carrion",No,No,Yes,No,No,Yes
kalatiira,sternidae,36-42,70-80,100-145,9.14,0.24,grey,white,B,Yes,red,red,May,August-September,1-3,both,both,"lakes,archipelago",fish,No,No,Yes,No,No,Yes
lapintiira,sternidae,33-37,66-77,90-130,8.97,0.20,grey,white,B,Yes,red,red,May,July-August,1-3,both,both,"archipelago,lakes,marshland",fish,No,No,Yes,No,No,Yes
suokukko,scolopacidae,25-26,46-49,90-130,6.73,0.36,dappled brown,white,C,No,dark-brown,orange,April-May,June-July,4,F,F,"marshland,wetlands","invertebrates,seeds",No,No,No,Yes,Yes,No
taivaanvuohi,scolopacidae,25-27,39-45,90-110,5.91,0.40,dappled brown,white,C,Yes,reddish-brown,greenish-grey,April-May,August-October,4,F,F,"marshland,coastal-meadows",invertebrates,No,Yes,No,No,Yes,No
lehtokurppa,scolopacidae,34-36,55-65,280-330,5.68,0.54,dappled brown,dappled beige,C,Yes,brown,yellowish-grey,March-April,September-October,4,F,F,forests,"invertebrates,worms,larvae,snails",No,Yes,No,No,Yes,No
metsäviklo,scolopacidae,21-24,39-44,75-85,7.20,0.29,brown,white,C,Yes,black,greenish-grey,March-April,June-July,4,both,M,"forests,ponds","invertebrates,plants",No,Yes,No,Yes,Yes,No


In [39]:
# Mappings to get more general groupings from the original group
second_level = {'laridae': 'lari', 'sternidae': 'lari', 'scolopacidae': 'charadrii', 'charadriidae': 'charadrii', 'haematopodidae': 'charadrii', 
                'dabbling ducks': 'anatinae', 'diving ducks': 'anatinae', 'gruifores': 'gruidae'} # The last one is a typo in the data I think?
highest_level = {'lari': 'charadriiformes', 'charadrii': 'charadriiformes', 'anatinae': 'anatidae', 'anserinae': 'anatidae', 
                 'gruidae': 'gruiformes', 'rallidae': 'gruiformes', 'accipitridae': 'accipitriformes'}

def avg_from_str(str): # Get mean from a string of form 'a-b' where a and b are numbers
  spl_str = str.split('-')
  if len(spl_str) > 1: 
    min_v, max_v = spl_str
    return (float(max_v)+float(min_v))/2
  else:
    return float(str)

full_data = raw_birddata.copy()

# Process ranges and get BMI and WSI instead of weight & wspan
full_data['length'] = full_data['length'].map(lambda x: avg_from_str(x))
full_data['wspan'] = full_data['wspan'].map(lambda x: avg_from_str(x))
full_data['weight'] = full_data['weight'].map(lambda x: avg_from_str(x))
full_data['BMI'] = full_data['weight'] / full_data['length']**2
full_data['WSI'] = full_data['wspan'] / full_data['length']
full_data.drop(['wspan', 'weight'], axis=1, inplace=True)

# Get average eggs
full_data['eggs'] = full_data['eggs'].map(lambda x: avg_from_str(x))

# Get family
full_data['family'] = full_data["group"].map(second_level).fillna(full_data["group"])
full_data

,group,length,AR,wload,back,belly,ftype,sim,billcol,legcol,arrives,leaves,eggs,incub,ccare,biotope,diet,diver,long-billed,webbed-feet,long-legs,wading-bird,plunge-dives,BMI,WSI,family
species,,,,,,,,,,,,,,,,,,,,,,,,,,
naurulokki,laridae,36.0,8.13,0.31,light grey,white,B,Yes,red,red,March-April,July,2.0,both,both,"lakes,sea-bays","fish,invertebrates,garbage",No,No,Yes,No,No,Yes,0.212191,2.569444,lari
harmaalokki,laridae,60.0,8.24,0.64,bluish grey,white,B,Yes,yellow,reddish,March-April,August-December,2.0,both,both,"lakes,sea-coast,marshland","fish,garbage,chicks,grain",No,No,Yes,No,No,Yes,0.291667,2.258333,lari
isolokki,laridae,65.5,8.24,0.66,bluish grey,white,B,Yes,yellow,reddish,December,March,3.0,both,both,"sea-coast, harbours","fish,eggs,chicks,garbage,carrion",No,No,Yes,No,No,Yes,0.326321,2.259542,lari
kalatiira,sternidae,39.0,9.14,0.24,grey,white,B,Yes,red,red,May,August-September,2.0,both,both,"lakes,archipelago",fish,No,No,Yes,No,No,Yes,0.080539,1.923077,lari
lapintiira,sternidae,35.0,8.97,0.20,grey,white,B,Yes,red,red,May,July-August,2.0,both,both,"archipelago,lakes,marshland",fish,No,No,Yes,No,No,Yes,0.089796,2.042857,lari
suokukko,scolopacidae,25.5,6.73,0.36,dappled brown,white,C,No,dark-brown,orange,April-May,June-July,4.0,F,F,"marshland,wetlands","invertebrates,seeds",No,No,No,Yes,Yes,No,0.169166,1.862745,charadrii
taivaanvuohi,scolopacidae,26.0,5.91,0.40,dappled brown,white,C,Yes,reddish-brown,greenish-grey,April-May,August-October,4.0,F,F,"marshland,coastal-meadows",invertebrates,No,Yes,No,No,Yes,No,0.147929,1.615385,charadrii
lehtokurppa,scolopacidae,35.0,5.68,0.54,dappled brown,dappled beige,C,Yes,brown,yellowish-grey,March-April,September-October,4.0,F,F,forests,"invertebrates,worms,larvae,snails",No,Yes,No,No,Yes,No,0.248980,1.714286,charadrii
metsäviklo,scolopacidae,22.5,7.20,0.29,brown,white,C,Yes,black,greenish-grey,March-April,June-July,4.0,both,M,"forests,ponds","invertebrates,plants",No,Yes,No,Yes,Yes,No,0.158025,1.844444,charadrii


In [40]:
empty_feature = None

# Migration time features
def month_to_num(str, dict):
  months = str.split('-')
  if len(months) == 1:
    return dict[months[0]]
  else:
    return (dict[months[1]] + dict[months[0]])/2.0
  
arrival_dict = {'December': 0, 'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11}
leaving_dict = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
                'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12}

def month_cmp(str, dict, cut1, cut2, action):
  if (month_to_num(str, dict) < cut1):
    return "early_" + action
  elif (month_to_num(str, dict) <= cut2):
    return "average_" + action
  else:
    return "late_" + action

full_data['arrives'] = full_data['arrives'].apply(lambda x: month_cmp(x, arrival_dict, 4, 5, "migration_arrival"))
full_data['leaves'] = full_data['leaves'].apply(lambda x: month_cmp(x, leaving_dict, 8, 9, "migration_leaving"))

# Binary features
full_data['sim'] = full_data['sim'].apply(lambda x: 'similar-looking_genders' if (x == 'Yes') else 'different-looking_genders')
full_data['diver'] = full_data['diver'].apply(lambda x: 'diver' if (x == 'Yes') else empty_feature)
full_data['long-billed'] = full_data['long-billed'].apply(lambda x: 'long-billed' if (x == 'Yes') else empty_feature)
full_data['webbed-feet'] = full_data['webbed-feet'].apply(lambda x: 'webbed-feet' if (x == 'Yes') else empty_feature)
full_data['long-legs'] = full_data['long-legs'].apply(lambda x: 'long-legs' if (x == 'Yes') else empty_feature)
full_data['wading-bird'] = full_data['wading-bird'].apply(lambda x: 'wading-bird' if (x == 'Yes') else empty_feature)
full_data['plunge-dives'] = full_data['plunge-dives'].apply(lambda x: 'plunge-dives' if (x == 'Yes') else empty_feature)

# Colour features
full_data['back'] = full_data['back'].apply(lambda x: x + '_back')
full_data['belly'] = full_data['belly'].apply(lambda x: x + '_belly')
full_data['billcol'] = full_data['billcol'].apply(lambda x: x + '_billcol')
full_data['legcol'] = full_data['legcol'].apply(lambda x: x + '_legcol')

# female male roles
full_data['incub'] = full_data['incub'].apply(lambda x: x + '_incub')
full_data['ccare'] = full_data['ccare'].apply(lambda x: x + '_ccare')

full_data


,group,length,AR,wload,back,belly,ftype,sim,billcol,legcol,arrives,leaves,eggs,incub,ccare,biotope,diet,diver,long-billed,webbed-feet,long-legs,wading-bird,plunge-dives,BMI,WSI,family
species,,,,,,,,,,,,,,,,,,,,,,,,,,
naurulokki,laridae,36.0,8.13,0.31,light grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,early_migration_arrival,early_migration_leaving,2.0,both_incub,both_ccare,"lakes,sea-bays","fish,invertebrates,garbage",None,None,webbed-feet,None,None,plunge-dives,0.212191,2.569444,lari
harmaalokki,laridae,60.0,8.24,0.64,bluish grey_back,white_belly,B,similar-looking_genders,yellow_billcol,reddish_legcol,early_migration_arrival,late_migration_leaving,2.0,both_incub,both_ccare,"lakes,sea-coast,marshland","fish,garbage,chicks,grain",None,None,webbed-feet,None,None,plunge-dives,0.291667,2.258333,lari
isolokki,laridae,65.5,8.24,0.66,bluish grey_back,white_belly,B,similar-looking_genders,yellow_billcol,reddish_legcol,early_migration_arrival,early_migration_leaving,3.0,both_incub,both_ccare,"sea-coast, harbours","fish,eggs,chicks,garbage,carrion",None,None,webbed-feet,None,None,plunge-dives,0.326321,2.259542,lari
kalatiira,sternidae,39.0,9.14,0.24,grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,average_migration_arrival,average_migration_leaving,2.0,both_incub,both_ccare,"lakes,archipelago",fish,None,None,webbed-feet,None,None,plunge-dives,0.080539,1.923077,lari
lapintiira,sternidae,35.0,8.97,0.20,grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,average_migration_arrival,early_migration_leaving,2.0,both_incub,both_ccare,"archipelago,lakes,marshland",fish,None,None,webbed-feet,None,None,plunge-dives,0.089796,2.042857,lari
suokukko,scolopacidae,25.5,6.73,0.36,dappled brown_back,white_belly,C,different-looking_genders,dark-brown_billcol,orange_legcol,average_migration_arrival,early_migration_leaving,4.0,F_incub,F_ccare,"marshland,wetlands","invertebrates,seeds",None,None,None,long-legs,wading-bird,None,0.169166,1.862745,charadrii
taivaanvuohi,scolopacidae,26.0,5.91,0.40,dappled brown_back,white_belly,C,similar-looking_genders,reddish-brown_billcol,greenish-grey_legcol,average_migration_arrival,average_migration_leaving,4.0,F_incub,F_ccare,"marshland,coastal-meadows",invertebrates,None,long-billed,None,None,wading-bird,None,0.147929,1.615385,charadrii
lehtokurppa,scolopacidae,35.0,5.68,0.54,dappled brown_back,dappled beige_belly,C,similar-looking_genders,brown_billcol,yellowish-grey_legcol,early_migration_arrival,late_migration_leaving,4.0,F_incub,F_ccare,forests,"invertebrates,worms,larvae,snails",None,long-billed,None,None,wading-bird,None,0.248980,1.714286,charadrii
metsäviklo,scolopacidae,22.5,7.20,0.29,brown_back,white_belly,C,similar-looking_genders,black_billcol,greenish-grey_legcol,early_migration_arrival,early_migration_leaving,4.0,both_incub,M_ccare,"forests,ponds","invertebrates,plants",None,long-billed,None,long-legs,wading-bird,None,0.158025,1.844444,charadrii


In [41]:
# Handle numerical features
eggvals = np.percentile(full_data['eggs'], [33.33, 66.67])
lvals = np.percentile(full_data['length'], [33.33, 66.67])
arvals = np.percentile(full_data['AR'], [33.33, 66.67])
wlvals = np.percentile(full_data['wload'], [33.33, 66.67])
bmivals = np.percentile(full_data['BMI'], [33.33, 66.67])
wsivals = np.percentile(full_data['WSI'], [33.33, 66.67])

def val_cmp(value, cut1, cut2, low, avg, high):
  if (value < cut1):
    return low
  elif (value <= cut2):
    return avg
  else:
    return high

full_data['eggs'] = full_data['eggs'].apply(lambda x: val_cmp(x, *eggvals, "few_eggs", "avg_egg_layage", "many_eggs"))
full_data['length'] = full_data['length'].apply(lambda x: val_cmp(x, *lvals, "short", "avg_length", "long"))
full_data['AR'] = full_data['AR'].apply(lambda x: val_cmp(x, *arvals, "small_AR", "avg_AR", "large_AR"))
full_data['wload'] = full_data['wload'].apply(lambda x: val_cmp(x, *wlvals, "small_wload", "avg_wload", "large_wload"))
full_data['BMI'] = full_data['BMI'].apply(lambda x: val_cmp(x, *bmivals, "skinny", "average_BMI", "robust"))
full_data['WSI'] = full_data['WSI'].apply(lambda x: val_cmp(x, *wsivals, "short-winged", "average-winged", "long-winged"))

full_data


,group,length,AR,wload,back,belly,ftype,sim,billcol,legcol,arrives,leaves,eggs,incub,ccare,biotope,diet,diver,long-billed,webbed-feet,long-legs,wading-bird,plunge-dives,BMI,WSI,family
species,,,,,,,,,,,,,,,,,,,,,,,,,,
naurulokki,laridae,avg_length,large_AR,small_wload,light grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,early_migration_arrival,early_migration_leaving,few_eggs,both_incub,both_ccare,"lakes,sea-bays","fish,invertebrates,garbage",None,None,webbed-feet,None,None,plunge-dives,skinny,long-winged,lari
harmaalokki,laridae,long,large_AR,avg_wload,bluish grey_back,white_belly,B,similar-looking_genders,yellow_billcol,reddish_legcol,early_migration_arrival,late_migration_leaving,few_eggs,both_incub,both_ccare,"lakes,sea-coast,marshland","fish,garbage,chicks,grain",None,None,webbed-feet,None,None,plunge-dives,average_BMI,long-winged,lari
isolokki,laridae,long,large_AR,avg_wload,bluish grey_back,white_belly,B,similar-looking_genders,yellow_billcol,reddish_legcol,early_migration_arrival,early_migration_leaving,few_eggs,both_incub,both_ccare,"sea-coast, harbours","fish,eggs,chicks,garbage,carrion",None,None,webbed-feet,None,None,plunge-dives,robust,long-winged,lari
kalatiira,sternidae,avg_length,large_AR,small_wload,grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,average_migration_arrival,average_migration_leaving,few_eggs,both_incub,both_ccare,"lakes,archipelago",fish,None,None,webbed-feet,None,None,plunge-dives,skinny,average-winged,lari
lapintiira,sternidae,avg_length,large_AR,small_wload,grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,average_migration_arrival,early_migration_leaving,few_eggs,both_incub,both_ccare,"archipelago,lakes,marshland",fish,None,None,webbed-feet,None,None,plunge-dives,skinny,long-winged,lari
suokukko,scolopacidae,short,avg_AR,small_wload,dappled brown_back,white_belly,C,different-looking_genders,dark-brown_billcol,orange_legcol,average_migration_arrival,early_migration_leaving,avg_egg_layage,F_incub,F_ccare,"marshland,wetlands","invertebrates,seeds",None,None,None,long-legs,wading-bird,None,skinny,average-winged,charadrii
taivaanvuohi,scolopacidae,short,small_AR,avg_wload,dappled brown_back,white_belly,C,similar-looking_genders,reddish-brown_billcol,greenish-grey_legcol,average_migration_arrival,average_migration_leaving,avg_egg_layage,F_incub,F_ccare,"marshland,coastal-meadows",invertebrates,None,long-billed,None,None,wading-bird,None,skinny,short-winged,charadrii
lehtokurppa,scolopacidae,avg_length,small_AR,avg_wload,dappled brown_back,dappled beige_belly,C,similar-looking_genders,brown_billcol,yellowish-grey_legcol,early_migration_arrival,late_migration_leaving,avg_egg_layage,F_incub,F_ccare,forests,"invertebrates,worms,larvae,snails",None,long-billed,None,None,wading-bird,None,average_BMI,average-winged,charadrii
metsäviklo,scolopacidae,short,avg_AR,small_wload,brown_back,white_belly,C,similar-looking_genders,black_billcol,greenish-grey_legcol,early_migration_arrival,early_migration_leaving,avg_egg_layage,both_incub,M_ccare,"forests,ponds","invertebrates,plants",None,long-billed,None,long-legs,wading-bird,None,skinny,average-winged,charadrii


In [42]:
# multivalue categorical features

data_split_biotope = full_data['biotope'].str.get_dummies(sep=',')
for c in data_split_biotope.columns:
  data_split_biotope[c] = data_split_biotope[c].apply(lambda x: 'habitat_' + c if x else empty_feature)

data_split_diet = full_data['diet'].str.get_dummies(sep=',')
for c in data_split_diet.columns:
  data_split_diet[c] = data_split_diet[c].apply(lambda x: 'eats_' + c if x else empty_feature)

data_final = pd.concat([full_data, data_split_biotope, data_split_diet], axis=1)
data_final.drop(['biotope', 'diet'], axis=1, inplace=True)

# Remove any remaining whitespace
data_final = data_final.replace(' ', '-', regex=True)

data_final.to_csv('data/final_birds.txt', header=None, index=None, sep=' ', mode='a')
data_final.to_csv('data/final_birds_comma.txt', header=None, index=None, sep=',', mode='a')
data_final

,group,length,AR,wload,back,belly,ftype,sim,billcol,legcol,arrives,leaves,eggs,incub,ccare,diver,long-billed,webbed-feet,long-legs,wading-bird,plunge-dives,BMI,WSI,family,forest-edges,harbours,archipelago,coastal-meadows,fells,fields,forest-edges,forests,islets,lakes,marshland,meadows,nutrient-rich-lakes,pastures,ponds,reedbeds,sea-bays,sea-coast,shores,shrub-tundra,streams,wetlands,algae,berries,birds,carrion,chicks,clams,eggs,fish,frogs,garbage,grain,grass,hares,insects,invertebrates,larvae,lizards,molluscs,plants,rodents,seeds,shellfish,small-rodents,snails,snakes,squirrels,vertebrae,worms
species,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
naurulokki,laridae,avg_length,large_AR,small_wload,light-grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,early_migration_arrival,early_migration_leaving,few_eggs,both_incub,both_ccare,None,None,webbed-feet,None,None,plunge-dives,skinny,long-winged,lari,None,None,None,None,None,None,None,None,None,habitat_lakes,None,None,None,None,None,None,habitat_sea-bays,None,None,None,None,None,None,None,None,None,None,None,None,eats_fish,None,eats_garbage,None,None,None,None,eats_invertebrates,None,None,None,None,None,None,None,None,None,None,None,None,None
harmaalokki,laridae,long,large_AR,avg_wload,bluish-grey_back,white_belly,B,similar-looking_genders,yellow_billcol,reddish_legcol,early_migration_arrival,late_migration_leaving,few_eggs,both_incub,both_ccare,None,None,webbed-feet,None,None,plunge-dives,average_BMI,long-winged,lari,None,None,None,None,None,None,None,None,None,habitat_lakes,habitat_marshland,None,None,None,None,None,None,habitat_sea-coast,None,None,None,None,None,None,None,None,eats_chicks,None,None,eats_fish,None,eats_garbage,eats_grain,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
isolokki,laridae,long,large_AR,avg_wload,bluish-grey_back,white_belly,B,similar-looking_genders,yellow_billcol,reddish_legcol,early_migration_arrival,early_migration_leaving,few_eggs,both_incub,both_ccare,None,None,webbed-feet,None,None,plunge-dives,robust,long-winged,lari,None,habitat_-harbours,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,habitat_sea-coast,None,None,None,None,None,None,None,eats_carrion,eats_chicks,None,eats_eggs,eats_fish,None,eats_garbage,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
kalatiira,sternidae,avg_length,large_AR,small_wload,grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,average_migration_arrival,average_migration_leaving,few_eggs,both_incub,both_ccare,None,None,webbed-feet,None,None,plunge-dives,skinny,average-winged,lari,None,None,habitat_archipelago,None,None,None,None,None,None,habitat_lakes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,eats_fish,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
lapintiira,sternidae,avg_length,large_AR,small_wload,grey_back,white_belly,B,similar-looking_genders,red_billcol,red_legcol,average_migration_arrival,early_migration_leaving,few_eggs,both_incub,both_ccare,None,None,webbed-feet,None,None,plunge-dives,skinny,long-winged,lari,None,None,habitat_archipelago,None,None,None,None,None,None,habitat_lakes,habitat_marshland,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,eats_fish,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
suokukko,scolopacidae,short,avg_AR,small_wload,dappled-brown_back,white_belly,C,different-looking_genders,dark-brown_billcol,orange_legcol,average_migration_arrival,early_migration_leaving,avg_egg_layage,F_incub,F_ccare,None,None,None,long-legs,wading-bird,None,skinny,average-winged,charadrii,None,None,None,None,None,None,None,None,None,None,habitat_marshland,None,None,None,None,None,None,None,None,None,None,habitat_wetlands,None,None,None,No